<a href="https://colab.research.google.com/github/gmazzitelli/GSN2/blob/master/MissioniDotazioni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**per poter essere lle il file in questo modo deve essere un google speradsheet**

quindi:
1. esportare i dati delle missioni come file excell
2. copiare ed incollare il file missioni nel tab dell'anno relativo sul google spreedsheet
3. selezionare nell'apertura del file il tab opportuno




In [1]:
import numpy as np
import pandas as pd
key = "1l-nUkzMRyt4gte4nkMcoBBzDZElnOIeBsGA8zIOVP_E" # foglio missioni
key = "1puUfSofNzvDKDYycQBy0NiKtWcMQSW9iwUDp_GtSXBk" # consuntivo 2019 - ricordarsi di salvare il file come google spreedsheet prima di copiare la chiave


def load_from_gspreadsheet(key, sheet_name):
    # google spredsheet TO url 
    # url = "https://docs.google.com/spreadsheet/ccc?key="+key+"&output=csv&sheet={Sheet2}"
    url = 'https://docs.google.com/spreadsheets/d/{key}/gviz/tq?tqx=out:csv&sheet={sheet_name}&headers=1'.format(
        key=key, sheet_name=sheet_name.replace(' ', '%20'))
    print(url)
    return url
def sp2pd(key, sheet_name, verbose=False):
    # spreadsheet TO Panda
    import pandas as pd
    df = pd.read_csv(load_from_gspreadsheet(key, sheet_name))
    # df = df[df.File_Number.isnull() == False]
    df.fillna('', inplace=True)
    if verbose: print ('Variables: ', df.columns.values)
    return df
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
plt.style.use('fast')
plt.rc('font', size = 14, family ='Sans-Serif')

In [23]:
anno = '2019'
df = sp2pd(key='1l-nUkzMRyt4gte4nkMcoBBzDZElnOIeBsGA8zIOVP_E', sheet_name=anno)
DOTAZIONE = sp2pd(key='1l-nUkzMRyt4gte4nkMcoBBzDZElnOIeBsGA8zIOVP_E', sheet_name='DOTAZIONE')

https://docs.google.com/spreadsheets/d/1l-nUkzMRyt4gte4nkMcoBBzDZElnOIeBsGA8zIOVP_E/gviz/tq?tqx=out:csv&sheet=2019&headers=1
https://docs.google.com/spreadsheets/d/1l-nUkzMRyt4gte4nkMcoBBzDZElnOIeBsGA8zIOVP_E/gviz/tq?tqx=out:csv&sheet=DOTAZIONE&headers=1


In [20]:
assegnazione = df[df.Doc=="Stanziamento iniziale (assegnazioni)"].Totale.str.replace('.', '').str.replace(',', '.').astype(float).sum()
speso = df[(df.Doc!="Stanziamento iniziale (assegnazioni)")].Totale.str.replace('.', '').str.replace(',', '.').astype(float).sum()
print ("Assegnato = ", assegnazione, "Speso = ", speso, "Disponibile = ", assegnazione-speso)
SpesoT=0.

Assegnato =  34000.0 Speso =  30812.000000000004 Disponibile =  3187.9999999999964


In [29]:
DOTAZIONE

,anno,esperimento,importo,totale
0,2019,DTZ,8500,
1,2019,KM3,3500,
2,2019,DARKSIDE,5000,
3,2019,JemEUSO,15000,
4,2019,LIMADOU_CSN2,2000,34000
5,2020,DTZ,12000,
6,2020,KM3,3000,
7,2020,DARKSIDE,0,15000
8,2021,DTZ,10000,
9,2021,KM3,2500,


In [34]:
DOTAZIONE.anno.values==int(anno)

array([ True,  True,  True,  True,  True, False, False, False, False,
       False, False])

In [41]:
DOTAZIONE[(DOTAZIONE.anno.values==int(anno)) & (DOTAZIONE.esperimento=='DTZ')].importo.values[0]

8500

In [21]:
df = df.drop(df.index[0])
df.Intestatario.value_counts()

RICCI MARCO                   38
NAPOLITANO TOMMASO            18
FRANCESCHI MASSIMO ALBERTO    11
MAZZITELLI GIOVANNI            7
PAOLONI ALESSANDRO             6
MODESTINO GIUSEPPINA           3
MARTINI AGNESE                 3
Votano Lucia                   2
Capone Antonio                 2
ORECCHINI DARIO                2
DE PAOLIS ALBERTO              2
DI GIOVANNI GIANNI             2
LIGI CARLO                     2
ANGELONI FABRIZIO              2
MENGUCCI ALESSANDRO            1
ROSATELLI FILIPPO              1
DELL'AGNELLO SIMONE            1
Name: Intestatario, dtype: int64

In [44]:
Darkside = DOTAZIONE[(DOTAZIONE.anno.values==int(anno)) & (DOTAZIONE.esperimento=='DARKSIDE')].importo.values[0]
assegnato = Darkside
user1 = (df.Intestatario=="NAPOLITANO TOMMASO") ^ (df.Intestatario=="FRANCESCHI MASSIMO ALBERTO")
speso = df[user1].Totale.str.replace('.', '').str.replace(',', '.').astype(float).sum()
print ("Speso Missioni = ",  speso, "Assegnato = ", assegnato, "Rimanete = ", assegnato-speso)
SpesoT+=speso
df[user1]


Speso Missioni =  3752.33 Assegnato =  5000 Rimanete =  1247.67


,Data,Tipo,Num,Doc,Intestatario,Descrizione,Assegn.,SubJudice,Iniziale,Variaz.,Totale
4,,Missioni,29267,Impegno contabile,FRANCESCHI MASSIMO ALBERTO,Laboratori Nazionali del Gran Sasso \ Meeting ...,"0,00","0,00","183,29","0,00","183,29"
5,,Missioni,29273,Impegno contabile,NAPOLITANO TOMMASO,Laboratori Nazionali del Gran Sasso \ Partecip...,"0,00","0,00","198,00","0,00","198,00"
7,,Missioni,29368,Impegno contabile,FRANCESCHI MASSIMO ALBERTO,Torrita di Siena \ Test di vibrazione esperime...,"0,00","0,00","494,55","0,00","494,55"
8,05/02/2019,Missioni,29431,Impegno contabile,NAPOLITANO TOMMASO,Torrita di Siena (SI) \ TEST DI VIBRAZIONE ESP...,"0,00","0,00","499,00","0,00","499,00"
12,11/03/2019,Missioni,29677,Impegno contabile,FRANCESCHI MASSIMO ALBERTO,L'Aquila \ General meeting esperimento DARKSID...,"0,00","0,00","351,60","0,00","351,60"
13,11/03/2019,Missioni,29678,Impegno contabile,FRANCESCHI MASSIMO ALBERTO,L'Aquila \ Technical Board esperimento DARKSID...,"0,00","0,00","176,60","0,00","176,60"
14,11/03/2019,Missioni,29683,Impegno contabile,NAPOLITANO TOMMASO,GSSI L'Aquila \ Partecipazione a General Meeti...,"0,00","0,00","376,00","0,00","376,00"
15,11/03/2019,Missioni,29684,Impegno contabile,NAPOLITANO TOMMASO,GSSI L'Aquila \ Partecipazione a Technical Boa...,"0,00","0,00","200,00","0,00","200,00"
18,10/04/2019,Missioni,29927,Impegno contabile,NAPOLITANO TOMMASO,CERN \ Meeting Tecnico su Integrazione Esperim...,"0,00","0,00","1.105,89","0,00","1.105,89"
25,,Missioni,30278,Impegno contabile,NAPOLITANO TOMMASO,Torrita di Siena (SI) \ TEST DI VIBRAZIONE ESP...,"0,00","0,00","289,00","0,00","289,00"


In [45]:
KM3 = DOTAZIONE[(DOTAZIONE.anno.values==int(anno)) & (DOTAZIONE.esperimento=='KM3')].importo.values[0]
assegnato = KM3
user2 = (df.Intestatario=="MARTINI AGNESE") ^ (df.Intestatario=="Capone Antonio")
speso = df[user2].Totale.str.replace('.', '').str.replace(',', '.').astype(float).sum()
print ("Speso Missioni = ",  speso, "Assegnato = ", assegnato, "Rimanete = ", assegnato-speso)
SpesoT+=speso
df[user2]


Speso Missioni =  2714.23 Assegnato =  3500 Rimanete =  785.77


,Data,Tipo,Num,Doc,Intestatario,Descrizione,Assegn.,SubJudice,Iniziale,Variaz.,Totale
1,,Missioni,29001,Impegno contabile,MARTINI AGNESE,Laboratori Nazionali del Sud \ riunione IB dat...,"0,00","0,00","7,65","0,00","7,65"
29,08/07/2019,Missioni,30624,Impegno contabile,MARTINI AGNESE,Varsavia \ collaboration meeting KM3NET data d...,"0,00","0,00","1.470,00","0,00","1.470,00"
32,11/07/2019,Missioni,30667,Impegno contabile,Capone Antonio,Varsavia \ partecipazione alle riunioni degli ...,"0,00","0,00","1.188,00","0,00","1.188,00"
82,08/07/2019,Missioni,30624,Variazione impegno,MARTINI AGNESE,Varsavia \ collaboration meeting KM3NET data d...,"0,00","0,00","0,00","-15,26","-15,26"
84,11/07/2019,Missioni,30667,Variazione impegno,Capone Antonio,Varsavia \ partecipazione alle riunioni degli ...,"0,00","0,00","0,00","63,84","63,84"


In [46]:
DTZ = DOTAZIONE[(DOTAZIONE.anno.values==int(anno)) & (DOTAZIONE.esperimento=='DTZ')].importo.values[0]
assegnato = DTZ
user3 = np.invert(user0 ^ user1 ^ user2)
speso = df[user3].Totale.str.replace('.', '').str.replace(',', '.').astype(float).sum()
print ("Speso Missioni = ",  speso, "Assegnato = ", assegnato, "Rimanete = ", assegnato-speso)
SpesoT+=speso
df[user3]


Speso Missioni =  40449.7 Assegnato =  8500 Rimanete =  -31949.699999999997


,Data,Tipo,Num,Doc,Intestatario,Descrizione,Assegn.,SubJudice,Iniziale,Variaz.,Totale
0,07/01/2019,,,Stanziamento iniziale (assegnazioni),,Stanziamento iniziale,"34.000,00","0,00","0,00","0,00","34.000,00"
11,07/03/2019,Missioni,29660,Impegno contabile,PAOLONI ALESSANDRO,Siena \ Riunione della commissione scientifica...,"0,00","0,00","680,00","0,00","680,00"
19,,Missioni,30035,Impegno contabile,MAZZITELLI GIOVANNI,Palma de Mallorca \ presentazioni alla confere...,"0,00","0,00","527,79","0,00","527,79"
22,,Missioni,30154,Impegno contabile,LIGI CARLO,Friburgo \ partecipazione a conferenza PATRAS ...,"0,00","0,00","1.000,00","0,00","1.000,00"
23,,Missioni,30197,Impegno contabile,DELL'AGNELLO SIMONE,San Francisco \ Riunioni e presentazioni con A...,"0,00","0,00","500,00","0,00","500,00"
31,10/07/2019,Missioni,30652,Impegno contabile,DI GIOVANNI GIANNI,CERN \ Open Day Cern data da 13-SEP-19 a 15-SE...,"0,00","0,00","215,00","0,00","215,00"
34,,Missioni,30688,Impegno contabile,PAOLONI ALESSANDRO,Laboratori Nazionali del Gran Sasso \ Partecip...,"0,00","0,00","154,67","0,00","154,67"
35,10/08/2019,Missioni,30813,Impegno contabile,MAZZITELLI GIOVANNI,Alghero \ ILS - Innovative learning School dat...,"0,00","0,00","350,00","0,00","350,00"
36,10/08/2019,Missioni,30814,Impegno contabile,MAZZITELLI GIOVANNI,Milano \ Referaggio Nu @ FNAL / ICARUS data da...,"0,00","0,00","600,00","0,00","600,00"
37,12/08/2019,Missioni,30815,Impegno contabile,PAOLONI ALESSANDRO,Milano \ Riunione per referaggio esperimenti I...,"0,00","0,00","500,00","0,00","500,00"


In [47]:
print ("Assegnato = ", DTZ+KM3+Darkside+Limadou+JemEUSO, "Speso = ", SpesoT, "Disponibile = ", DTZ+KM3+Darkside+Limadou+JemEUSO-SpesoT)


Assegnato =  34000 Speso =  85421.97 Disponibile =  -51421.97


In [43]:
Limadou = DOTAZIONE[(DOTAZIONE.anno.values==int(anno)) & (DOTAZIONE.esperimento=='LIMADOU_CSN2')].importo.values[0]
JemEUSO = DOTAZIONE[(DOTAZIONE.anno.values==int(anno)) & (DOTAZIONE.esperimento=='JemEUSO')].importo.values[0]
assegnato = Limadou+JemEUSO
user0 = (df.Intestatario=="RICCI MARCO") ^ (df.Intestatario=="DE PAOLIS ALBERTO")
speso = df[user0].Totale.str.replace('.', '').str.replace(',', '.').astype(float).sum()
print ("Speso Missioni = ",  speso, "Assegnato = ", assegnato, "Rimanete = ", assegnato-speso)
SpesoT+=speso
df[user0]

Speso Missioni =  17895.739999999998 Assegnato =  17000 Rimanete =  -895.739999999998


,Data,Tipo,Num,Doc,Intestatario,Descrizione,Assegn.,SubJudice,Iniziale,Variaz.,Totale
2,07/01/2019,Missioni,29181,Impegno contabile,RICCI MARCO,Laboratori Nazionali di Legnaro \ LNL 10-11 e ...,"0,00","0,00","218,80","0,00","218,80"
3,08/01/2019,Missioni,29209,Impegno contabile,RICCI MARCO,CERN \ Presentazione al CERN Recognized Experi...,"0,00","0,00","770,70","0,00","770,70"
6,,Missioni,29322,Impegno contabile,RICCI MARCO,Trento Institute for Fundamental Physics and A...,"0,00","0,00","475,70","0,00","475,70"
9,,Missioni,29560,Impegno contabile,RICCI MARCO,Sezione di Roma Tor Vergata \ Riunione analisi...,"0,00","0,00","39,30","0,00","39,30"
10,,Missioni,29603,Impegno contabile,RICCI MARCO,Sezione di Roma Tor Vergata \ Esecuzione test ...,"0,00","0,00","67,90","0,00","67,90"
16,,Missioni,29805,Impegno contabile,RICCI MARCO,Servizio di Presidenza \ Incontro con Presiden...,"0,00","0,00","18,00","0,00","18,00"
17,03/04/2019,Missioni,29882,Impegno contabile,RICCI MARCO,Siena \ Partecipazione con presentazione alla ...,"0,00","0,00","365,95","0,00","365,95"
20,,Missioni,30045,Impegno contabile,RICCI MARCO,Sezione di Roma Tor Vergata \ Riunione analisi...,"0,00","0,00","19,30","0,00","19,30"
21,,Missioni,30146,Impegno contabile,RICCI MARCO,LODZ \ Partecipazione a General Collaboration ...,"0,00","0,00","1.434,20","0,00","1.434,20"
24,,Missioni,30277,Impegno contabile,DE PAOLIS ALBERTO,Torrita di Siena \ Test Vibrazione Esperimento...,"0,00","0,00","273,39","0,00","273,39"
